In [8]:
from thinkbayes import Pmf
from thinkbayes import Suite

In [2]:
#쿠키 문제
pmf = Pmf()
pmf.Set('Bowl 1', 0.5)
pmf.Set('Bowl 2', 0.5)

pmf.Mult('Bowl 1', 0.75)
pmf.Mult('Bowl 2', 0.5)

pmf.Normalize()

pmf.Prob('Bowl 1')

0.6000000000000001

In [16]:
#베이지안 프레임워크: 쿠키 문제
class Cookie(Pmf):
    def __init__(self, hypos):
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()
    
    mixes = {
        'Bowl 1':dict(vanilla=0.75, chocolate=0.25),
        'Bowl 2':dict(vanilla=0.5, chocolate=0.5),
        }
    def Likelihood(self, data, hypo):
        """The likelihood of the data under the hypothesis.

        data: string cookie type
        hypo: string bowl ID
        """
        mix = self.mixes[hypo]
        like = mix[data]
        return like

    def Update(self, data):
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self.Normalize()

hypos = ['Bowl 1', 'Bowl 2']

pmf = Cookie(hypos)

pmf.Update('vanilla')

for hypo, prob in pmf.Items():
    print(hypo, prob)

dataset = ['vanilla', 'chocolate', 'vanilla']
for data in dataset:
    pmf.Update(data)
    for hypo, prob in pmf.Items():
        print(hypo, prob)

Bowl 1 0.6000000000000001
Bowl 2 0.4
Bowl 1 0.6923076923076923
Bowl 2 0.30769230769230765
Bowl 1 0.5294117647058825
Bowl 2 0.4705882352941177
Bowl 1 0.627906976744186
Bowl 2 0.37209302325581395


In [7]:
#베이지안 프레임워크: 몬티홀 문제 1

class Monty(Pmf):
    def __init__(self, hypos):
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()
    
    def Likelihood(self, data, hypo):
        if hypo == data: return 0
        elif hypo == 'A': return 0.5
        else: return 1

    def Update(self, data):
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self.Normalize()
hypos = 'ABC'
pmf =  Monty(hypos)
data = 'B'
pmf.Update(data)
for hypo, prob in pmf.Items():
    print(hypo, prob)

A 0.3333333333333333
B 0.0
C 0.6666666666666666


In [10]:
#베이지안 프레임워크: 몬티홀, 캡슐화 in thinkbayes.py
# class Suite(Pmf):
#     def __init__(self, hypo = tuple()):
#     """initialize a Suite object"""
#     def Update(self, data):
#     """update hypothesis based on data"""
class Monty(Suite):
    def Likelihood(self, data, hypo):
        if hypo == data: return 0
        elif hypo == 'A': return 0.5
        else: return 1

suite = Monty('ABC')
suite.Update('B')
suite.Print()

A 0.3333333333333333
B 0.0
C 0.6666666666666666


In [15]:
#베이지안 프레임워크: M&M
class M_and_M(Suite):
    mix94 = dict(brown = 30, yellow = 20, red = 20, green = 10, orange = 10, tan = 10)
    mix96 = dict(blue = 24, green = 20, orange = 16, yellow = 14, red = 13, brown = 13)

    hypoA = dict(bag1=mix94 , bag2=mix96)
    hypoB = dict(bag1=mix96 , bag2=mix94)
    hypotheses = dict(A=hypoA, B = hypoB)
    def Likelihood(self, data, hypo):
        bag, color = data
        mix = self.hypotheses[hypo][bag]
        like = mix[color]
        return like
suite = M_and_M('AB')
suite.Update(('bag1', 'yellow'))
suite.Update(('bag2', 'green'))
suite.Print()

A 0.7407407407407407
B 0.2592592592592592
